In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import uproot

In [2]:
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [3]:
path_to_data = '../data/'

folder15 = 'breynold/user.breynold.data15_13TeV.00284484.physics_Main.DAOD_NTUP_JTRIG_JETM1.r9264_p3083_p3601_j042_tree.root/'
file15 = 'user.breynold.18753218._000001.tree.root'
folder16 = 'breynold/user.breynold.data16_13TeV.00307656.physics_Main.DAOD_NTUP_JTRIG_JETM1.r9264_p3083_p3601_j042_tree.root/'
file16 = 'user.breynold.18797259._000001.tree.root'

folder = 'doglioni/'
file = 'data18_13TeV.00355331.calibration_DataScouting_05_Jets.merge.AOD.r10657_p3592.root'
file = 'dijetISRphoton.root'
file = 'dijetISRphoton36k.root'

# Load a ROOT file
filePath = path_to_data + folder16 + file16
#filePath = path_to_data + folder + file
ttree = uproot.open(filePath)['outTree']['nominal']
#ttree = uproot.open(filePath)['CollectionTree']

In [4]:
ttree.keys()

[b'runNumber',
 b'eventNumber',
 b'lumiBlock',
 b'coreFlags',
 b'bcid',
 b'NPV',
 b'actualInteractionsPerCrossing',
 b'averageInteractionsPerCrossing',
 b'weight_pileup',
 b'correctedAverageMu',
 b'correctedAndScaledAverageMu',
 b'correctedActualMu',
 b'correctedAndScaledActualMu',
 b'passL1',
 b'passHLT',
 b'passedTriggers',
 b'disabledTriggers',
 b'triggerPrescales',
 b'isPassBits',
 b'isPassBitsNames',
 b'nAntiKt4EMTopoJets_Calib2016',
 b'AntiKt4EMTopoJets_Calib2016_E',
 b'AntiKt4EMTopoJets_Calib2016_pt',
 b'AntiKt4EMTopoJets_Calib2016_phi',
 b'AntiKt4EMTopoJets_Calib2016_eta',
 b'AntiKt4EMTopoJets_Calib2016_clean_passLooseBad',
 b'AntiKt4EMTopoJets_Calib2016_clean_passTightBad',
 b'AntiKt4EMTopoJets_Calib2016_JvtPass_Medium',
 b'nAntiKt4EMTopoJets_Calib2018',
 b'AntiKt4EMTopoJets_Calib2018_E',
 b'AntiKt4EMTopoJets_Calib2018_pt',
 b'AntiKt4EMTopoJets_Calib2018_phi',
 b'AntiKt4EMTopoJets_Calib2018_eta',
 b'AntiKt4EMTopoJets_Calib2018_clean_passLooseBad',
 b'AntiKt4EMTopoJets_Calib201

In [5]:
branchnames = ['nAntiKt4EMTopoJets_Calib2018',
               'AntiKt4EMTopoJets_Calib2018_E',
               'AntiKt4EMTopoJets_Calib2018_pt',
               'AntiKt4EMTopoJets_Calib2018_phi',
               'AntiKt4EMTopoJets_Calib2018_eta']


#branchnames = ['nDSTrigJet',
#               'DSTrigJet_E',
#              'DSTrigJet_pt',
#              'DSTrigJet_phi',
#              'DSTrigJet_eta']

#branchnames = ['n_jet',
#               'jet_e',
#               'jet_pt',
#               'jet_phi',
#               'jet_eta',
#               'n_photons',
#               'photons_e',
#               'photons_pt',
#               'photons_phi',
#               'photons_eta']

jaggedE = ttree.array(branchnames[1])
jaggedpT = ttree.array(branchnames[2])
jaggedphi = ttree.array(branchnames[3])
jaggedeta = ttree.array(branchnames[4])

In [6]:
# Remove events with no jets
jaggedE = jaggedE[(jaggedE.counts != 0)]
jaggedpT = jaggedpT[(jaggedpT.counts != 0)]
jaggedphi = jaggedphi[(jaggedphi.counts != 0)]
jaggedeta = jaggedeta[(jaggedeta.counts != 0)]

In [7]:
print(jaggedE.shape, jaggedpT.shape, jaggedphi.shape ,jaggedeta.shape)

(1939900,) (1939900,) (1939900,) (1939900,)


In [8]:
# Get indices that would sort array according to leading jet pT
argsort = np.array(jaggedpT[:,0]).argsort()

In [9]:
jaggedpT = jaggedpT[argsort]
jaggedE = jaggedE[argsort]
jaggedeta = jaggedeta[argsort]
jaggedphi = jaggedphi[argsort]

In [10]:
path_to_save_folder = '../data/all_jets_split_data/'

N_datasets = 4
split_index = len(jaggedpT) // N_datasets

for n in range(N_datasets):
    sub_jaggedpT = jaggedpT[n * split_index:(n + 1) * split_index]
    sub_jaggedeta = jaggedeta[n * split_index:(n + 1) * split_index]
    sub_jaggedphi = jaggedphi[n * split_index:(n + 1) * split_index]
    sub_jaggedE = jaggedE[n * split_index:(n + 1) * split_index]
    
    print(sub_jaggedpT[0][0])
    print(sub_jaggedpT[-1][0])

 #   e_list = []
#    pt_list = []
   # eta_list = []
  #  phi_list = []
 #   for ii, arr in enumerate(sub_jaggedE):
#        for kk, val in enumerate(arr):

        #    e_list.append(val)
       #     pt_list.append(sub_jaggedpT[ii][kk])
      #      eta_list.append(sub_jaggedeta[ii][kk])
     #       phi_list.append(sub_jaggedphi[ii][kk])


    #all_e = np.array(e_list)
    #all_pt = np.array(pt_list)
    #all_eta = np.array(eta_list)
    #all_phi = np.array(phi_list)
    
    #df = pd.DataFrame({'pT': all_pt, 'eta': all_eta, 'phi': all_phi, 'E': all_e})
    #train, test = train_test_split(df, test_size=0.2, random_state=41)
    #train.to_pickle(path_to_save_folder + 'sub_train_all_jets_%d' % n)
    #test.to_pickle(path_to_save_folder + 'sub_test_all_jets_%d' % n)


7.4080515
161.86812
161.86838
207.1685
207.16852
279.61777
279.6179
20721.586
